In [5]:
#input amygdala volume and mask and outputs nifti files of amygdala for each kernel value and moment
from __future__ import print_function 
import sys
import os
import logging
import six
import nrrd
import numpy as np
import SimpleITK as sitk
from radiomics import featureextractor, getFeatureClasses
import radiomics

In [6]:
#input variables and path defs
workDir = ***Insert working directory here***
outDir = str(workDir+'outputs/')
volDir = str(workDir+'volumes/')

ses='01'
side='left'
side2='L'

In [ ]:
for i in [1,4,5,6,7,9,10,11,12,13]:
    if i <10:
        sub=str('PNC00'+str(i))
    else:
        sub=str('PNC0'+str(i))
    print(sub)
    #for loop that makes .nii files for every feature extracted from original volume at kernels 2,4,6,8,10
    featureClasses = getFeatureClasses()

    #imageName, maskName = radiomics.getTestCase('brain1')
    #amygdala volume
    im = sitk.ReadImage(volDir+'amyg_'+sub+'_'+ses+'_'+side+'_vol.nii.gz')
    #amygdala mask
    mask = sitk.ReadImage(outDir + 'amyg_mask_'+sub+'_'+ses+'_'+side+'_roi.nii.gz')
    
    #if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
    #    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

    # Set up and run extractor
    # Use hardcoded settings (separate for settings, input image types and enabled features)
    settings = {}
    settings['normalize'] = True
    settings['normalizeScale'] = 100
    settings['removeOutliers'] = 1
    settings['binWidth'] = 25
    settings['resampledPixelSpacing'] = None
    settings['interpolator'] = 4
    settings['verbose'] = True
    settings['voxelBatch'] = 10
    settings['correctMask'] = True
    # for loop goes through each kernel size for each feature
    for kernelRad in range(0, 5, 1):   
        settings['kernelRadius'] = kernelRad + 1
        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

        # Disable all classes
        extractor.disableAllFeatures()

        # Enable all features in firstorder
        #extractor.enableFeatureClassByName('firstorder')
        extractor.enableFeaturesByName(firstorder=['Mean', 'Variance', 'Skewness', 'Kurtosis'])

        # By default, only 'Original' (no filter applied) is enabled. Optionally enable some image types:
        extractor.disableAllImageTypes()
        extractor.enableImageTypeByName('Original')
        # extractor.enableImageTypeByName('Wavelet')
        # extractor.enableImageTypeByName('LoG', customArgs={'sigma':[4]})
        # extractor.enableImageTypeByName('Square')
        # extractor.enableImageTypeByName('SquareRoot')
        # extractor.enableImageTypeByName('Exponential')
        # extractor.enableImageTypeByName('Logarithm')
        print('Enabled input images:')
        for imageType in extractor.enabledImagetypes.keys():
            print('\t' + imageType)

        # Calculate features
        print('Calculating features')
        result = extractor.execute(im, mask, voxelBased=True)

        # Show output voxel based
        outPath = str(outDir+sub+'_'+ses+'features_'+side2+'/')
        #make proper directory is it does not exist yet:
        import os
        if not os.path.exists(outPath):
            os.mkdir(outPath)
        for key, val in six.iteritems(result):
            if isinstance(val, sitk.Image):  # Feature map
                sitk.WriteImage(val, outPath + key + '_' + str(kernelRad + 1) + '.nrrd', True)
                print("Stored feature %s in %s" % (key, key + ".nrrd"))
            else:  # Diagnostic info
                print("\t%s: %s" %(key, val))

        # Write to nifti
        imPath = outPath
        prefix = 'original_firstorder'

        # Read images and save as nii
        volItk = sitk.ReadImage(imPath + prefix + '_Mean_' + str(kernelRad + 1) + '.nrrd')
        sitk.WriteImage(volItk, imPath + prefix + '_Mean_' + str(kernelRad + 1) + '_1sd.nii.gz')

        volItk = sitk.ReadImage(imPath + prefix + '_Variance_' + str(kernelRad + 1) + '.nrrd')
        sitk.WriteImage(volItk, imPath + prefix + '_Variance_' + str(kernelRad + 1) + '_1sd.nii.gz')

        volItk = sitk.ReadImage(imPath + prefix + '_Skewness_' + str(kernelRad + 1) + '.nrrd')
        sitk.WriteImage(volItk, imPath + prefix + '_Skewness_' + str(kernelRad + 1) + '_1sd.nii.gz')

        volItk = sitk.ReadImage(imPath + prefix + '_Kurtosis_' + str(kernelRad + 1) + '.nrrd')
        sitk.WriteImage(volItk, imPath + prefix + '_Kurtosis_' + str(kernelRad + 1) + '_1sd.nii.gz')

